In [0]:
from PIL import Image

path = "/Volumes/demo_catalog/bronze/raw_files/conceitos_chave_unity_catalog.png"
img = Image.open(path)
display(img)

## Criando o primeiro Catalogo de Dados

In [0]:
%sql

CREATE CATALOG IF NOT EXISTS demo_catalog
COMMENT 'Catálogo de demonstração criado para o workhop de Unity Catalog';

## Criando SCHEMAS (databases) no catalogodemo_catalog

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS demo_catalog.bronze
COMMENT 'Schema bronze para dados brutos e heterogêneos';

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS demo_catalog.silver
COMMENT 'Schema silver para dados tratados';

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS demo_catalog.gold
COMMENT 'Schema gold para dados que serão utilizados por negócio';

## Criando uma tabela

In [0]:
%sql

CREATE TABLE IF NOT EXISTS demo_catalog.bronze.sales(
  sales_id INT,
  product STRING,
  quantity INT,
  price DOUBLE,
  sale_date DATE
)
USING DELTA
COMMENT 'Tabela de vendas brutas no formato Delta';

In [0]:
%sql

INSERT INTO demo_catalog.bronze.sales (sales_id, product, quantity, price, sale_date)
VALUES
  -- Grupo 1: Notebook
  (1, 'Notebook Dell', 1, 5500.00, DATE '2025-10-01'),
  (2, 'Notebook Dell', 2, 5200.00, DATE '2025-10-02'),
  (3, 'Notebook Dell', 1, 5300.00, DATE '2025-10-03'),
  (4, 'Notebook Dell', 3, 5400.00, DATE '2025-10-04'),
  (5, 'Notebook Dell', 1, 5600.00, DATE '2025-10-05'),
  -- Grupo 2: Monitor
  (6, 'Monitor LG 27"', 2, 1200.00, DATE '2025-10-06'),
  (7, 'Monitor LG 27"', 1, 1150.00, DATE '2025-10-07'),
  (8, 'Monitor LG 27"', 3, 1180.00, DATE '2025-10-08'),
  (9, 'Monitor LG 27"', 2, 1220.00, DATE '2025-10-09'),
  (10, 'Monitor LG 27"', 1, 1250.00, DATE '2025-10-10'),
  -- Grupo 3: Headset
  (11, 'Headset HyperX', 2, 600.00, DATE '2025-10-11'),
  (12, 'Headset HyperX', 1, 590.00, DATE '2025-10-12'),
  (13, 'Headset HyperX', 3, 620.00, DATE '2025-10-13'),
  (14, 'Headset HyperX', 2, 610.00, DATE '2025-10-14'),
  (15, 'Headset HyperX', 1, 630.00, DATE '2025-10-15')

In [0]:
%sql

SELECT * FROM demo_catalog.bronze.sales

## Criando uma view

In [0]:
%sql

CREATE OR REPLACE VIEW demo_catalog.bronze.vw_sales_summary AS
    SELECT
      product,
      SUM(quantity) AS total_sold,
      ROUND(SUM(quantity * price),2) AS revenue
    FROM demo_catalog.bronze.sales
    GROUP BY product;

In [0]:
%sql

SELECT * FROM demo_catalog.bronze.vw_sales_summary;

## Criando um volume

In [0]:
%sql

CREATE VOLUME demo_catalog.bronze.raw_files
COMMENT 'Volume para armazenar arquivos brutos de ingestão incial (planilhas, csv, imagens, json, etc)';

In [0]:
from PIL import Image

path = "/Volumes/demo_catalog/bronze/raw_files/gatinho.png"
img = Image.open(path)
display(img)

In [0]:
print(img.format, img.size, img.mode)

In [0]:
%pip install torch torchvision

In [0]:
import torch
from torchvision import models, transforms
import urllib.request
from PIL import Image

#Caminho da imagem
path = "/Volumes/demo_catalog/bronze/raw_files/gatinho.png"

#Carregando rótulos do ImageNet
labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_labels = urllib.request.urlopen(labels_url).read().decode("utf-8").splitlines()

# Preprocessamento padrão do ResNet
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Carregar imagem e preparar Tensor
input_tensor = preprocess(Image.open(path).convert("RGB")).unsqueeze(0)

# Modelo pré-treinado
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.eval()

#Inferência
with torch.no_grad():
    logits = model(input_tensor)
    probs = torch.nn.functional.softmax(logits, dim=1)[0]

topk = torch.topk(probs, k=5)
topk5 =  [(imagenet_labels[i], float(probs[i])) for i in topk.indices.tolist()]
topk5

### Tranformando resultados de predição em tabela

In [0]:
from pyspark.sql import Row
import json

prediction = Row(
    file_path = path,
    top1_label = topk5[0][0],
    top1_prob = topk5[0][1],
    top5 = json.dumps(topk5)
)

spark.createDataFrame([prediction])\
    .write.mode("append")\
    .saveAsTable("demo_catalog.bronze.image_predictions")

In [0]:
%sql

SELECT * FROM demo_catalog.bronze.image_predictions

## Trabalhando com arquivos csv, json e parquet

### Ler e exibir dados

In [0]:
# === Parquet ===
df_parquet = spark.read.parquet("/Volumes/demo_catalog/bronze/raw_files/dados.parquet")
print("🟢 PARQUET:")
df_parquet.show(truncate=False)

# === CSV ===
df_csv = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/demo_catalog/bronze/raw_files/dados.csv")
)
print("🟢 CSV:")
df_csv.show(truncate=False)

# === JSON ===
df_json = spark.read.json("/Volumes/demo_catalog/bronze/raw_files/dados.json")
print("🟢 JSON:")
df_json.show(truncate=False)


### Fazer queries SQL com o DataFrame API do PySpark

In [0]:
# Exemplo de SELECT * (totos os dados)
df_parquet.select("*").show()

# Exemplo de SELECT especifico
df_parquet.select("id", "nome", "salario").show()

# Exemplo de SELECT com condicional
df_parquet.filter(df_parquet.salario < 9000).select("nome", "salario").show()

### Podemos salvar volumes em views temporárias

createOrReplaceTempView, criam views temporárias e não vão para o Catalog por ser temporárias.

In [0]:
# Ler Parquet
df_parquet = spark.read.parquet("/Volumes/demo_catalog/bronze/raw_files/dados.parquet")
df_parquet.createOrReplaceTempView("vw_parquet")

# Ler CSV
df_csv = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/demo_catalog/bronze/raw_files/dados.csv")
)
df_csv.createOrReplaceTempView("vw_csv")

# Ler JSON
df_json = spark.read.json("/Volumes/demo_catalog/bronze/raw_files/dados.json")
df_json.createOrReplaceTempView("vw_json")

## Criar uma funções SQL no Unity Catalog

### Ideal quando a lógica é simples (ex: cálculo, concatenação, formatação)

In [0]:
%sql
-- Functions é uma forma de gravar KPIs (Funções personalizadas que não existem no sql)
CREATE OR REPLACE FUNCTION demo_catalog.bronze.calc_bonus(salario DOUBLE, percentual DOUBLE)
RETURNS DOUBLE
COMMENT 'Calcula bônus de acordo com o percentual informado'
RETURN salario * (percentual / 100.0);

In [0]:
%sql
-- Sempre ao usar uma função ou fazer alguma coisa que eu não queria ficar especificando qual catalogo e schema, eu posso rodar esse comando aqui
USE CATALOG demo_catalog;
USE SCHEMA bronze;

In [0]:
%sql

SELECT
  nome,
  salario,
  calc_bonus(salario, 10) AS bonus
FROM vw_parquet;

# Criar um modelo via MLFlow

In [0]:
%sql
-- 1) Garanta que o catalogo e o schema existem
CREATE CATALOG IF NOT EXISTS demo_catalog;
CREATE SCHEMA IF NOT EXISTS demo_catalog.ml
COMMENT 'Schema para registrar modelo de ML (Unity Catalog)';

In [0]:
# %pip install mlflow scikit-learn --quiet  # se precisar

import pandas as pd
import mlflow, mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature

# === Dados como DataFrame (facilita a signature com nomes de colunas) ===
iris = load_iris(as_frame=True)
X, y = iris.data, iris.target  # X é DataFrame, y é Series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# === Modelo ===
clf = RandomForestClassifier(n_estimators=50, random_state=42).fit(X_train, y_train)

# === Signature + input_example (obrigatórios para UC Models) ===
input_example = X_train.iloc[:3]
signature = infer_signature(X_train, clf.predict(X_train))

# === Registrar no Unity Catalog ===
mlflow.set_registry_uri("databricks-uc")

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="model",
        registered_model_name="demo_catalog.ml.iris_model",  # <catalog>.<schema>.<name>
        signature=signature,
        input_example=input_example
    )

print("✅ Modelo registrado com signature: demo_catalog.ml.iris_model")
